In [ ]:
import torch
import time
import copy
import os
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset
import torchvision
from torchvision.datasets import ImageFolder
from torch.utils.data import random_split
from torchvision import datasets, transforms, models
from PIL import Image


#basic config
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
batch_size = 32
num_classes = 101
input_size = 224
feature_extract = True
num_epochs = 10
learning_rate = 0.001
# Paths

data_root = "./food-101/images"
train_meta = "./food-101/meta/train.txt"
test_meta = "./food-101/meta/test.txt"

#Loading and modification of model EfficientNetV2-M
model_ft = models.efficientnet_v2_m(weights=models.EfficientNet_V2_M_Weights.DEFAULT)

# Congela tutti i layer tranne il classificatore
for param in model_ft.features.parameters():
    param.requires_grad = False

# Modifica il classificatore finale
model_ft.classifier[1] = nn.Linear(model_ft.classifier[1].in_features, num_classes)
model_ft = model_ft.to(device)

#Ottimizzatore, loss e lancio del training
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(filter(lambda p: p.requires_grad, model.parameters()), lr=learning_rate)



trainloader = torch.utils.data.DataLoader(train_data, batch_size=128, shuffle=True)
testloader = torch.utils.data.DataLoader(test_data, batch_size=128)
test_loader=torch.utils.data.DataLoader(testdata, batch_size=64)

prog_schedule = [
    {'image_size': 128, 'epochs': 10, 'aug_strength': 'light', 'dropout': 0.1},
    {'image_size': 160, 'epochs': 10, 'aug_strength': 'medium', 'dropout': 0.2},
    {'image_size': 224, 'epochs': 20, 'aug_strength': 'strong', 'dropout': 0.3},
]


mean = np.array([0.485, 0.456, 0.406])
std = np.array([0.229, 0.224, 0.225])


class Food101Dataset(Dataset):
    def __init__(self, root_dir, meta_file, transform=None):
        self.root_dir = root_dir
        self.transform = transform

        # Read the meta file lines
        with open(meta_file, 'r') as f:
            lines = f.read().splitlines()

        self.samples = []
        self.class_to_idx = {}
        idx = 0
        for line in lines:
            # line like 'apple_pie/997124'
            cls, img_id = line.split('/')
            if cls not in self.class_to_idx:
                self.class_to_idx[cls] = idx
                idx += 1
            img_path = os.path.join(root_dir, cls, img_id + '.jpg')
            self.samples.append((img_path, self.class_to_idx[cls]))

    def __len__(self):
        return len(self.samples)

    def __getitem__(self, idx):
        img_path, label = self.samples[idx]
        image = Image.open(img_path).convert('RGB')
        if self.transform:
            image = self.transform(image)
        return image, label




#Preprocessing
def get_train_transforms(image_size, aug_strength):
    aug_list = [transforms.RandomHorizontalFlip()]
    if aug_strength in ['medium', 'strong']:
        aug_list += [transforms.ColorJitter(0.2, 0.2, 0.2, 0.1)]
    if aug_strength == 'strong':
        aug_list += [transforms.RandomRotation(30)]
    return transforms.Compose([
        transforms.Resize((image_size, image_size)),
        *aug_list,
        transforms.ToTensor(),
        transforms.Normalize(mean, std),
    ])

def get_test_transforms(image_size):
    return transforms.Compose([
        transforms.Resize((image_size, image_size)),
        transforms.ToTensor(),
        transforms.Normalize(mean, std),
    ])


#DATABASE LOADING (CAMBIA DIR)

# Load using custom dataset class
full_train_dataset = Food101Dataset(data_root, train_meta, transform=None)
test_dataset = Food101Dataset(data_root, test_meta, transform=None)


train_size = int(0.9 * len(full_train_dataset))
val_size = len(full_train_dataset) - train_size
train_dataset, val_dataset = random_split(full_train_dataset, [train_size, val_size])


def train_model(model, dataloaders, criterion, optimizer, num_epochs):
    since = time.time()
    best_model_wts = copy.deepcopy(model.state_dict())
    best_acc = 0.0

    for epoch in range(num_epochs):
        print(f"Epoch {epoch+1}/{num_epochs}")

        for phase in ['train', 'test']:
            if phase == 'train':
                model.train()
            else:
                model.eval()

            running_loss = 0.0
            running_corrects = 0

            for inputs, labels in dataloaders[phase]:
                inputs = inputs.to(device)
                labels = labels.to(device)

                optimizer.zero_grad()

                with torch.set_grad_enabled(phase == 'train'):
                    outputs = model(inputs)
                    _, preds = torch.max(outputs, 1)
                    loss = criterion(outputs, labels)

                    if phase == 'train':
                        loss.backward()
                        optimizer.step()

                running_loss += loss.item() * inputs.size(0)
                running_corrects += torch.sum(preds == labels.data)

            epoch_loss = running_loss / len(dataloaders[phase].dataset)
            epoch_acc = running_corrects.double() / len(dataloaders[phase].dataset)

            print(f"{phase} Loss: {epoch_loss:.4f} Acc: {epoch_acc:.4f}")

            if phase == 'test' and epoch_acc > best_acc:
                best_acc = epoch_acc
                best_model_wts = copy.deepcopy(model.state_dict())

        print()

    time_elapsed = time.time() - since
    print(f"Training complete in {time_elapsed // 60:.0f}m {time_elapsed % 60:.0f}s")
    print(f"Best test Acc: {best_acc:.4f}")

    model.load_state_dict(best_model_wts)
    return model

# Progressive learning stages
prog_schedule = [
    {'image_size': 128, 'epochs': 10, 'aug_strength': 'light', 'dropout': 0.1},
    {'image_size': 160, 'epochs': 10, 'aug_strength': 'medium', 'dropout': 0.2},
    {'image_size': 224, 'epochs': 20, 'aug_strength': 'strong', 'dropout': 0.3},
]

# Load EfficientNetV2
model_ft = models.efficientnet_v2_m(weights=models.EfficientNet_V2_M_Weights.DEFAULT)
model_ft.classifier[1] = nn.Linear(model_ft.classifier[1].in_features, num_classes)
model_ft = model_ft.to(device)

criterion = nn.CrossEntropyLoss()

# Progressive training loop
for stage in prog_schedule:
    image_size = stage['image_size']
    aug_strength = stage['aug_strength']
    dropout = stage['dropout']
    print(f"Training with image size {image_size} and augmentation {aug_strength}")


    # Update dropout dynamically
    model_ft.classifier[0].p = dropout

    # Update transforms dynamically
    train_dataset.dataset.transform = get_train_transforms(image_size, aug_strength)
    val_dataset.dataset.transform = get_test_transforms(image_size)
    test_dataset.transform = get_test_transforms(image_size)

    dataloaders_dict = {
        'train': DataLoader(train_dataset, batch_size=32, shuffle=True, num_workers=4),
        'val': DataLoader(val_dataset, batch_size=32, shuffle=False, num_workers=4),
        'test': DataLoader(test_dataset, batch_size=32, shuffle=False, num_workers=4)
    }


    optimizer = optim.Adam(filter(lambda p: p.requires_grad, model_ft.parameters()), lr=learning_rate)

    # Train model
    model_ft = train_model(model_ft, dataloaders_dict, criterion, optimizer, num_epochs=stage['epochs'])

# Save the final model
torch.save(model_ft.state_dict(), 'efficientnetv2_food101_progressive.pth')

# After training is done
test_loader = dataloaders_dict['test']
model_ft.eval()

correct = 0
total = 0
with torch.no_grad():
    for images, labels in test_loader:
        images = images.to(device)
        labels = labels.to(device)
        outputs = model_ft(images)
        _, predicted = torch.max(outputs, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print(f"Final Test Accuracy: {correct / total:.4f}")
